# imports for all methods

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from training import EvalLandmark
import CERDataset

torch.cuda.empty_cache()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dtype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor
print('CUDA available: {}'.format(torch.cuda.is_available()))

# mode settings for mode = 'pretrain2'
mode = 'pretrain2'
source_dir = 'TrainingImages2' # check
source_name = 'image_'
landmarks_dir = 'TrainingLabels1' # check
landmarks_name = 'labels_'
target_dir = 'DetectedLandmarks2' # check
target_name = 'imageDetected_'
params_name = '2pretrain_params.pt' # check
train_loss_name = '2pretrain_train_loss.npy' #???
val_loss_name = '2pretrain_val_loss.npy' #???
loss_fig = '2loss_Pretraining.png' # check
loss_file = '2loss.txt' # check

# landmarks
numOfLandmarks = 4

CUDA available: True


# single run - no search

In [ ]:

from torch.utils.data import Dataset, DataLoader
from nni.retiarii.evaluator.pytorch import Lightning, Trainer
from cnn6 import CNN

# mode settings for mode = 'pretrain2'
mode = 'pretrain2'
source_dir = 'TrainingImages2' # check
source_name = 'image_'
landmarks_dir = 'TrainingLabels1' # check
landmarks_name = 'labels_'
target_dir = 'DetectedLandmarks2' # check
target_name = 'imageDetected_'
params_name = '2pretrain_params.pt' # check
train_loss_name = '2pretrain_train_loss.npy' #???
val_loss_name = '2pretrain_val_loss.npy' #???
loss_fig = '2loss_Pretraining.png' # check
loss_file = '2loss.txt' # check

# landmarks
numOfLandmarks = 4

# settings for pytorch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.empty_cache()
torch.set_default_dtype(torch.float64)

model = CNN()
model.to(device)

# tinker with weight and bias definitions here
# new cnn6 design labels the layers slightly differently
depth = 6
new_state_dict = {}
old_weights = torch.load(params_name,map_location=torch.device(device))
for i in range(depth):  # Assuming depth is the number of convolutional layers.
    new_state_dict[f"convs.{i}.weight"] = old_weights[f"conv{i + 1}.weight"]
    new_state_dict[f"convs.{i}.bias"] = old_weights[f"conv{i + 1}.bias"]
new_state_dict["fullyConnected.weight"] = old_weights["fullConnected7.weight"]
new_state_dict["fullyConnected.bias"] = old_weights["fullConnected7.bias"]
new_state_dict["output.weight"] = old_weights["output.weight"]
new_state_dict["output.bias"] = old_weights["output.bias"]

model.load_state_dict(new_state_dict)

# load and shuffle data and define train/test dataloader
full_data = CERDataset.CERDataset(landmark_dir=landmarks_dir, image_dir=source_dir)
train_data_size = int(0.72 * len(full_data))
test_data_size = int(0.18 * len(full_data))
valid_data_size = len(full_data) - train_data_size - test_data_size
assert train_data_size + test_data_size + valid_data_size == len(full_data)
train_data, test_data, valid_data = torch.utils.data.random_split(full_data,
                                                                  [train_data_size, test_data_size, valid_data_size],
                                                                  generator=torch.Generator().manual_seed(42))

train_dataloader = DataLoader(train_data, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=8, shuffle=True)
valid_dataloader = DataLoader(valid_data, batch_size=8, shuffle=False)

module = EvalLandmark(mode=mode)


# from lightning.pytorch.callbacks.early_stopping import EarlyStopping
# early_stop_callback = EarlyStopping(
#             monitor="validation_loss", 
#             min_delta=0.00, 
#             patience=200, 
#             verbose=False, 
#             mode="max"
#             )

trainer = Trainer(
            max_epochs=10000,
            fast_dev_run=False,
            gpus=1,
            enable_progress_bar=False,
            # callbacks=[early_stop_callback],
            )


lightning = Lightning(lightning_module=module, trainer=trainer, train_dataloaders=train_dataloader, val_dataloaders=valid_dataloader)
lightning.fit(model)

# multi-trial search

In [ ]:
# MT specific imports
from nni import trace
import nni.retiarii.strategy as strategy
from nni.retiarii.experiment.pytorch import RetiariiExperiment, RetiariiExeConfig
from nni.retiarii.evaluator.pytorch import Lightning, Trainer
from nni.retiarii.evaluator.pytorch.lightning import DataLoader
from trainingMT import EvalLandmarkMT
from cnn6_space_mt import CNN_Space_MT



# load and shuffle data and define train/test dataloader
full_data = CERDataset.CERDataset(landmark_dir=landmarks_dir, image_dir=source_dir)
train_data_size = int(0.72 * len(full_data))
test_data_size = int(0.18 * len(full_data))
valid_data_size = len(full_data) - train_data_size - test_data_size
assert train_data_size + test_data_size + valid_data_size == len(full_data)
train_data, test_data, valid_data = torch.utils.data.random_split(full_data,
                                                                  [train_data_size, test_data_size, valid_data_size],
                                                                  generator=torch.Generator().manual_seed(42))

train_dataloader = DataLoader(train_data, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=8, shuffle=True)
valid_dataloader = DataLoader(valid_data, batch_size=8, shuffle=False)

# Create a Lightning Module
module = EvalLandmarkMT(mode=mode)

# Create a Search Space
# this implementation gives flexibility in the depth of the encoder half of the network
model_space = CNN_Space_MT(attention=True, depth=6)

# configure the trainer
trainer = Trainer(
            max_epochs=1500,
            fast_dev_run=False,
            gpus=1,
            enable_progress_bar=False,
            )

# # Select a Search Strategy
# https://nni.readthedocs.io/en/latest/nas/exploration_strategy.html
search_strategy = strategy.Random(dedup=True)
# search_strategy = strategy.TPE()
# search_strategy = strategy.RegularizedEvolution(dedup=True)


lightning = Lightning(lightning_module=module, trainer=trainer, train_dataloaders=train_dataloader, val_dataloaders=valid_dataloader)

# Configure and run the experiment for multi-strategy
experiment = RetiariiExperiment(model_space, lightning, [], search_strategy)
exp_config = RetiariiExeConfig('local')
exp_config.experiment_name = f'MT_{search_strategy.__class__.__name__}_LandmarkDetection'

# exp_config.max_trial_number = 250   # spawn {input} trials at most
exp_config.trial_concurrency = 3  # will run {input} trials concurrently # on my machine each trial took up about 3260MiB of GPU memory so adjust accordingly
# you can update both of these values in the web UI if you want to run more trials concurrently or run more trials in total

exp_config.trial_gpu_number = 1
exp_config.training_service.use_active_gpu = True

experiment.run(exp_config, 8889) # recommended in documentation was 8081 but 8889 worked ony my machine

In [ ]:
exported_arch = experiment.export_top_models()

exported_arch

In [ ]:
experiment.stop()

# oneshot search

In [2]:
# oneshot search strategies evaluate concurrently by weight sharing
# the search space needs to be restricted as the gpu memory gets full quickly in these methods
# I recommend using a few different trials with:
#     - short depth and wide range of convolution choices (kernals, dilation, depthwise, etc)
#     - long depth and narrow range of convolution choices

# the oneshot model will also need longer training time than a single trial

# documentation: https://colab.research.google.com/github/microsoft/nni/blob/1037f585678280e9242ed1ad73bca7c00e7a0eab/docs/source/tutorials/darts.ipynb#scrollTo=FK0fC9fE5kr6

import nni.retiarii.strategy as strategy
from nni.retiarii.experiment.pytorch import RetiariiExperiment, RetiariiExeConfig
from nni.retiarii.evaluator.pytorch import Lightning, Trainer
from nni.retiarii.evaluator.pytorch.lightning import DataLoader
from cnn6_space_os import CNN_Space_OS
from trainingOS import EvalLandmarkOS


# load and shuffle data and define train/test dataloader
full_data = CERDataset.CERDataset(landmark_dir=landmarks_dir, image_dir=source_dir)
train_data_size = int(0.72 * len(full_data))
test_data_size = int(0.18 * len(full_data))
valid_data_size = len(full_data) - train_data_size - test_data_size
assert train_data_size + test_data_size + valid_data_size == len(full_data)
train_data, test_data, valid_data = torch.utils.data.random_split(full_data,
                                                                  [train_data_size, test_data_size, valid_data_size],
                                                                  generator=torch.Generator().manual_seed(42))

train_dataloader = DataLoader(train_data, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=8, shuffle=True)
valid_dataloader = DataLoader(valid_data, batch_size=8, shuffle=False)

# Create a Lightning Module
module = EvalLandmarkOS(mode=mode)

# configure the trainer
trainer = Trainer(
            max_epochs=1500,
            fast_dev_run=False,
            gpus=1,
            enable_progress_bar=False,
            )

if not hasattr(trainer, 'optimizer_frequencies'):
    trainer.optimizer_frequencies = []
    
lightning = Lightning(lightning_module=module, trainer=trainer, train_dataloaders=train_dataloader, val_dataloaders=valid_dataloader)

# Select the Search Strategy
search_strategy = strategy.DARTS()
# search_strategy = strategy.DartsStrategy()
# search_strategy = strategy.ENAS()
# search_strategy = strategy.GumbelDARTS()
# search_strategy = strategy.RandomOneShot()

# Create a Search Space
# this implementation gives flexibility in the depth of the encoder half of the network
model_space = CNN_Space_OS(attention=False, depth=1)

config = RetiariiExeConfig(execution_engine='oneshot')
experiment = RetiariiExperiment(model_space, evaluator=lightning, strategy=search_strategy)
experiment.run(config)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


KeyError: '_nni_symbol'

In [ ]:
exported_arch = experiment.export_top_models()

exported_arch

In [ ]:
# stop experiment and clear cache
experiment.stop()
torch.cuda.empty_cache()